# 第3章：SubAgent实现 - 多Agent协作

## 本章目标

本章将实现SubAgent机制，让主Agent能够创建专门的子Agent来处理复杂任务。你将学习：

- **SubAgent配置**: 定义不同类型的专用Agent
- **TaskTool设计**: 创建调用SubAgent的工具
- **上下文隔离**: 确保SubAgent独立运行
- **工具过滤**: 为不同SubAgent分配不同工具
- **结果合成**: 将SubAgent结果返回给主Agent

## 应用场景

- **代码分析**: 创建专门的代码审查SubAgent
- **文档撰写**: 创建专门的技术写作SubAgent
- **数据分析**: 创建专门的数据处理SubAgent
- **并发任务**: 多个SubAgent并行处理不同子任务


## 💡 这是 Claude Code 的什么功能？

这是 Claude Code 的**专业分工机制**。当遇到复杂任务时，主 Agent 会调用专门的子 Agent：

**典型场景**：
- 用户："帮我分析这个 10 万行的代码库"
- 主 Agent："这个任务太大了，我调用 **代码分析专家** 来处理"
- 代码分析专家：独立分析，不被主对话干扰，完成后返回摘要

**优势**：
- **上下文隔离**（子 Agent 看不到主对话的无关内容）
- **工具专用**（可以给子 Agent 配置专门的工具集）
- **并行处理**（未来可以同时运行多个子 Agent）

**技术要点**：
- 使用独立的 Agent 实例
- 通过 TaskTool 调用子 Agent
- 每个子 Agent 有独立的消息历史

本章实现的 SubAgent 架构，就是这个功能的核心。

## 1. 环境设置

In [ ]:
import os
from typing import TypedDict, List, Optional, Dict
from IPython.display import Image, display

# LangGraph
from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState

# LangChain
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage

# LLM
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatTongyi

In [ ]:
# 配置
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "chapter-03-subagent"

# 初始化LLM
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 选项2: 通义千问 (取消注释使用)
llm = ChatTongyi(model="qwen3-max", temperature=0)
print(f"✅ LLM初始化成功: {llm.model_name}")

## 2. 定义基础工具集

首先定义一组基础工具，这些工具将被分配给不同的SubAgent。

In [ ]:
@tool
def read_file(filepath: str) -> str:
    """读取文件内容（模拟）
    
    Args:
        filepath: 文件路径
    """
    # 模拟读取文件
    mock_files = {
        "example.py": "def hello():\n    print('Hello World')",
        "readme.md": "# Project Documentation\nThis is a sample project."
    }
    return mock_files.get(filepath, f"文件 {filepath} 不存在")


@tool
def search_code(query: str) -> str:
    """搜索代码（模拟）
    
    Args:
        query: 搜索关键词
    """
    return f"找到包含 '{query}' 的代码：\n- example.py:1\n- utils.py:15"


@tool
def analyze_complexity(code: str) -> str:
    """分析代码复杂度（模拟）
    
    Args:
        code: 要分析的代码
    """
    return f"复杂度分析：循环复杂度=3, 代码行数={len(code.split())*2}"


@tool
def write_file(filepath: str, content: str) -> str:
    """写入文件（模拟）
    
    Args:
        filepath: 文件路径
        content: 文件内容
    """
    return f"已写入文件 {filepath}，内容长度：{len(content)} 字符"


@tool
def format_markdown(text: str) -> str:
    """格式化Markdown文档（模拟）
    
    Args:
        text: 原始文本
    """
    return f"格式化后的Markdown:\n{text.upper()}"


# 所有基础工具
base_tools = [
    read_file,
    search_code,
    analyze_complexity,
    write_file,
    format_markdown
]

print(f"✅ 定义了 {len(base_tools)} 个基础工具")
for t in base_tools:
    print(f"  - {t.name}")

## 3. SubAgent配置

定义不同类型的SubAgent，每个都有自己的系统提示词和允许使用的工具。

In [ ]:
class SubAgentConfig(TypedDict):
    """SubAgent配置"""
    type: str  # SubAgent类型
    system_prompt: str  # 系统提示词
    allowed_tools: Optional[List[str]]  # 允许的工具名称列表，None表示所有工具


# SubAgent配置列表
subagent_configs: List[SubAgentConfig] = [
    {
        "type": "general-purpose",
        "system_prompt": """你是一个通用AI助手，擅长处理各种复杂的多步骤任务。
        
你的特点：
- 擅长文件搜索、内容分析和代码理解
- 系统化地分解复杂问题
- 提供详细准确的分析结果

请认真分析用户需求，使用可用工具完成任务。""",
        "allowed_tools": None  # 所有工具都可用（除了TaskTool）
    },
    {
        "type": "code-analyzer",
        "system_prompt": """你是代码分析专家，专注于：

- 代码质量评估和改进建议
- 架构设计分析和优化
- 性能瓶颈识别和解决方案
- 安全漏洞检测和修复建议

请提供具体、可操作的技术建议。""",
        "allowed_tools": ["read_file", "search_code", "analyze_complexity"]
    },
    {
        "type": "document-writer",
        "system_prompt": """你是技术文档撰写专家，专注于：

- 清晰准确的技术文档
- 用户友好的操作指南
- 完整的API文档和示例
- 结构化的项目文档

确保文档的可读性和实用性。""",
        "allowed_tools": ["read_file", "write_file", "format_markdown"]
    }
]

print(f"✅ 定义了 {len(subagent_configs)} 种SubAgent类型：")
for config in subagent_configs:
    tools_desc = "所有工具" if config["allowed_tools"] is None else f"{len(config['allowed_tools'])}个工具"
    print(f"  - {config['type']}: {tools_desc}")

## 4. 创建SubAgent实例

为每种类型创建独立的Agent实例，确保它们有独立的执行环境。

In [ ]:
# 存储不同类型的Agent实例
agent_instances: Dict[str, any] = {}

# 为每种类型创建Agent
for config in subagent_configs:
    # 过滤工具
    if config["allowed_tools"] is None:
        # 所有工具都可用（后续会排除TaskTool）
        filtered_tools = base_tools
    else:
        # 只允许指定的工具
        allowed_names = set(config["allowed_tools"])
        filtered_tools = [t for t in base_tools if t.name in allowed_names]
    
    # 创建Agent
    agent = create_react_agent(
        llm,
        tools=filtered_tools,
        prompt=config["system_prompt"]
    )
    
    agent_instances[config["type"]] = agent
    print(f"✅ 创建了 {config['type']} Agent，工具数: {len(filtered_tools)}")

print(f"\n共创建 {len(agent_instances)} 个SubAgent实例")

## 5. 创建TaskTool

TaskTool是主Agent调用SubAgent的接口。

In [ ]:
from enum import Enum

# 创建SubAgent类型的枚举（用于工具参数验证）
SubAgentType = Enum(
    'SubAgentType',
    {config['type'].replace('-', '_'): config['type'] for config in subagent_configs}
)

@tool
def task_tool(description: str, subagent_type: str) -> str:
    """启动专用SubAgent处理复杂的多步骤任务
    
    
    使用规则：
    1. 任务复杂且需要专业处理时使用此工具
    2. 每次SubAgent调用是独立且无状态的
    3. 提供详细的任务描述，SubAgent会自主完成
    4. SubAgent完成后，请向用户总结关键信息
    
    何时使用SubAgent：
    - 复杂的多步骤分析任务
    - 需要专业技能的任务（代码分析、文档撰写）
    - 大规模文件搜索和处理
    - 可并行处理的独立任务
    
    Args:
        description: SubAgent的详细任务描述
        subagent_type: 要使用的SubAgent类型
        
    Returns:
        SubAgent的执行结果
    """
    # 验证SubAgent类型
    if subagent_type not in agent_instances:
        available = ", ".join(agent_instances.keys())
        return f"错误：未知的SubAgent类型 '{subagent_type}'。可用类型：{available}"
    
    # 获取对应的Agent
    agent = agent_instances[subagent_type]
    
    try:
        print(f"\n🤖 启动SubAgent [{subagent_type}] from task_tool")
        print(f"📋 任务: {description}")
        
        # 创建隔离的执行上下文
        # 重要：每个SubAgent调用都是独立的，有自己的消息历史
        result = agent.invoke({
            "messages": [HumanMessage(content=description)]
        })
        
        # 提取最终响应
        final_message = result["messages"][-1]
        response_content = final_message.content
        
        print(f"✅ SubAgent [{subagent_type}] 完成")
        
        # 返回格式化结果
        formatted_result = f"""SubAgent [{subagent_type}] 执行完成：

任务: {description}

结果:
{response_content}

注意：此结果由专用SubAgent生成，请根据需要向用户总结关键信息。"""
        # print('subgent result:', formatted_result)
        return formatted_result
        
    except Exception as e:
        print(f"❌ SubAgent [{subagent_type}] 失败: {e}")
        return f"SubAgent [{subagent_type}] 执行失败: {str(e)}"


print("✅ TaskTool创建完成")

## 6. 创建主Agent

主Agent可以调用TaskTool来创建SubAgent。

In [ ]:
# 主Agent的工具：基础工具 + TaskTool
# 注意：基础工具数量较少，复杂任务应该委托给SubAgent
main_agent_tools = [
    read_file,  # 主Agent也可以读文件
    task_tool   # 关键：可以创建SubAgent
]

# 创建主Agent
main_agent = create_react_agent(
    llm,
    tools=main_agent_tools,
    prompt="""你是一个智能助手，负责协调和分配任务。

你的职责：
1. 理解用户需求
2. 判断任务复杂度
3. 对于复杂或专业任务，使用task_tool创建专用SubAgent
4. 汇总SubAgent结果并呈现给用户

何时使用SubAgent：
- 任务需要深入的代码分析 → code-analyzer
- 任务需要撰写技术文档 → document-writer
- 任务复杂且多步骤 → general-purpose

简单任务可以自己处理，不必创建SubAgent。
"""
)

print("✅ 主Agent创建完成")

In [ ]:
# 可视化主Agent
try:
    display(Image(main_agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"无法显示: {e}")

## 7. 测试SubAgent系统

### 测试1: 代码分析任务

In [ ]:
print("="*60)
print("测试1: 代码分析任务")
print("="*60)

result = main_agent.invoke({
    "messages": [HumanMessage(
        content="请分析 example.py 文件的代码质量，并给出优化建议"
    )]
})

print("\n📝 对话历史:")
print("="*60)
for i, msg in enumerate(result["messages"]):
    print(f"\n[{i+1}] {msg.__class__.__name__}:")
    if hasattr(msg, 'content') and msg.content:
        print(msg.content[:300] + "..." if len(msg.content) > 300 else msg.content)

### 测试2: 文档撰写任务

In [ ]:
print("="*60)
print("测试2: 文档撰写任务")
print("="*60)

result = main_agent.invoke({
    "messages": [HumanMessage(
        content="请为 example.py 中的代码编写API文档"
    )]
})

print("\n📝 最终回答:")
print("="*60)
result["messages"][-1].pretty_print()

### 测试3: 简单任务（不需要SubAgent）

In [ ]:
print("="*60)
print("测试3: 简单任务")
print("="*60)

result = main_agent.invoke({
    "messages": [HumanMessage(
        content="请读取 readme.md 文件的内容"
    )]
})

print("\n📝 最终回答:")
print("="*60)
result["messages"][-1].pretty_print()

## 8. 深入理解：上下文隔离

每个SubAgent都有完全独立的执行环境。

In [ ]:
# 演示上下文隔离
print("演示SubAgent的上下文隔离")
print("="*60)

# 主Agent的第一轮对话
result1 = main_agent.invoke({
    "messages": [HumanMessage(content="我的项目名称是 MyAwesomeProject")]
})

print("主Agent记住了项目名称")

# 第二轮：主Agent记得之前的内容
result2 = main_agent.invoke({
    "messages": result1["messages"] + [HumanMessage(content="我刚才说的项目叫什么？")]
})

print("\n主Agent的回答:")
result2["messages"][-1].pretty_print()

# 但如果通过SubAgent询问，它不会知道
print("\n" + "="*60)
print("通过SubAgent询问：")

result3 = main_agent.invoke({
    "messages": result2["messages"] + [
        HumanMessage(content="使用general-purpose SubAgent回答：我刚才说的项目叫什么？")
    ]
})

print("\nSubAgent的回答（它不知道之前的对话）:")
result3["messages"][-1].pretty_print()

注意：最终回答可能会答对项目名称，但这是根据早期对话获得的信息，你可以通过下面方式查看subagent返回的信息。

In [ ]:
# 查看result3['messages'][-2]的信息，这是在Task tool中调用subAgent返回的信息
print(result3['messages'][-2].content)

由上面输出可知，subagent确实不知道之前的对话

## 9. 工具过滤演示

In [ ]:
# 演示不同SubAgent有不同的工具访问权限
print("不同SubAgent的工具权限")
print("="*60)

for agent_type, agent in agent_instances.items():
    # 获取对应的配置
    config = next(c for c in subagent_configs if c["type"] == agent_type)
    
    print(f"\n{agent_type}:")
    if config["allowed_tools"] is None:
        print("  工具: 所有基础工具")
    else:
        print(f"  工具: {', '.join(config['allowed_tools'])}")
    
    # 测试：让code-analyzer尝试写文件（它没有这个权限）
    if agent_type == "code-analyzer":
        print("\n  测试：尝试写文件（应该失败）")
        result = agent.invoke({
            "messages": [HumanMessage(content="请写入文件 test.txt，内容是'Hello'")]
        })
        print(f"  结果: {result['messages'][-1].content[:100]}...")

## 10. 核心概念总结

### SubAgent架构

```
主Agent
  ├─ 分析任务
  ├─ 调用TaskTool
  │   ├─ 创建SubAgent实例
  │   ├─ 传递任务描述
  │   ├─ SubAgent独立执行
  │   └─ 返回结果
  └─ 汇总结果给用户
```

### 关键设计原则

1. **上下文隔离**: 每个SubAgent有独立的消息历史
2. **工具过滤**: 根据SubAgent类型限制可用工具
3. **防止递归**: SubAgent不能访问TaskTool
4. **职责分离**: 主Agent负责协调，SubAgent负责执行

### 工具分配策略

| SubAgent类型 | 允许的工具 | 使用场景 |
|--------------|-----------|----------|
| general-purpose | 所有工具 | 复杂多步骤任务 |
| code-analyzer | read_file, search_code, analyze_complexity | 代码分析 |
| document-writer | read_file, write_file, format_markdown | 文档撰写 |


## 11. 练习题

### 练习1: 添加新的SubAgent类型
创建一个 `data-analyst` SubAgent，配置合适的工具和提示词。

### 练习2: 结果缓存
为TaskTool添加缓存机制，相同的任务不重复执行。

### 练习3：动态发现Subagent
修改main_agent发现subagents的方式，当定义新的subagent时，main_agent能动态发现。

### 思考题
1. 为什么SubAgent不能访问TaskTool？
2. 如何实现SubAgent之间的通信？

## 12. 下一步

在下一章（第4章），我们将学习：
- 扩展State定义
- TodoRead/TodoWrite工具
- LLM自主任务管理
- 提示词工程的重要性